In [1]:
#Bibliotécas
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
#DataFrame
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']

df = pd.read_csv('adult.data.csv', names=headers)

In [3]:
#Tratamento de dados faltantes para o mais representatívo
columns = df.columns
for i in columns:
    missing = df[i].isin([' ?']).sum()
    df[i] = df[i].replace(' ?', np.NaN)
df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [4]:
#Divisão em Parâmetro e Classe
X_df = df.iloc[:, 0:14].values
y_df = df.iloc[:, 14].values

In [5]:
#LabelEncoder
def labelencoder(pd_serie):
    labelencoder = LabelEncoder()
    pd_serie = labelencoder.fit_transform(pd_serie)
    return pd_serie

X_df[:, 1] = labelencoder(X_df[:, 1])
X_df[:, 3] = labelencoder(X_df[:, 3])
X_df[:, 5] = labelencoder(X_df[:, 5])
X_df[:, 6] = labelencoder(X_df[:, 6])
X_df[:, 7] = labelencoder(X_df[:, 7])
X_df[:, 8] = labelencoder(X_df[:, 8])
X_df[:, 9] = labelencoder(X_df[:, 9])
X_df[:, 13] = labelencoder(X_df[:, 13])
y_df = labelencoder(y_df)

In [6]:
#One Hot Encoder
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])], remainder='passthrough')
X_df = onehotencorder.fit_transform(X_df).toarray()

In [7]:
#Standard Scaler
scaler = StandardScaler()
X_df = scaler.fit_transform(X_df)

In [8]:
#Divisão df de treinamento e teste 15%
X_df_train, X_df_test, y_df_train, y_df_test =  train_test_split(X_df, y_df, test_size=0.15, random_state=0)

In [9]:
print(X_df_train.shape, X_df_test.shape, y_df_train.shape, y_df_test.shape)

(27676, 105) (4885, 105) (27676,) (4885,)


In [10]:
#kNN
from sklearn.neighbors import KNeighborsClassifier

In [11]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_df_train, y_df_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [12]:
predict_knn = knn.predict(X_df_test)

In [13]:
accuracy_score(y_df_test, predict_knn)

0.828863868986694

In [14]:
confusion_matrix(y_df_test, predict_knn)

array([[3439,  254],
       [ 582,  610]])

In [15]:
print(classification_report(y_df_test, predict_knn))

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      3693
           1       0.71      0.51      0.59      1192

    accuracy                           0.83      4885
   macro avg       0.78      0.72      0.74      4885
weighted avg       0.82      0.83      0.82      4885



In [16]:
#cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True)
score_knn = cross_val_score(knn, X_df, y_df, cv = kfold)
score_knn.mean()

0.8293354928602088

In [40]:
#hold
knn_hold = knn.fit(X_df, y_df)

In [17]:
#random forest
from sklearn.ensemble import RandomForestClassifier

In [18]:
random_forest = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_leaf=1, min_samples_split=5, random_state=0)
random_forest.fit(X_df_train, y_df_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
predict_rf = random_forest.predict(X_df_test)

In [20]:
accuracy_score(y_df_test, predict_rf)

0.8534288638689866

In [21]:
confusion_matrix(y_df_test, predict_rf)

array([[3430,  263],
       [ 453,  739]])

In [22]:
print(classification_report(y_df_test, predict_rf))

              precision    recall  f1-score   support

           0       0.88      0.93      0.91      3693
           1       0.74      0.62      0.67      1192

    accuracy                           0.85      4885
   macro avg       0.81      0.77      0.79      4885
weighted avg       0.85      0.85      0.85      4885



In [23]:
kfold = KFold(n_splits=10, shuffle=True)
score_random_forest = cross_val_score(random_forest, X_df, y_df, cv = kfold)
score_random_forest.mean()

0.8584809395601536

In [41]:
#hold
random_forest_hold = random_forest.fit(X_df, y_df)

In [24]:
'''#MLP
import keras
from keras.models import Sequential
from keras.layers import Dense'''

'#MLP\nimport keras\nfrom keras.models import Sequential\nfrom keras.layers import Dense'

In [25]:
'''mlp_keras = Sequential()
mlp_keras.add(Dense(units=55, activation='relu', input_dim=108))
mlp_keras.add(Dense(units=55, activation='relu'))
mlp_keras.add(Dense(units=1, activation='sigmoid'))
mlp_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])'''

"mlp_keras = Sequential()\nmlp_keras.add(Dense(units=55, activation='relu', input_dim=108))\nmlp_keras.add(Dense(units=55, activation='relu'))\nmlp_keras.add(Dense(units=1, activation='sigmoid'))\nmlp_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])"

In [26]:
'''mlp_keras.fit(X_df_train, y_df_train, batch_size=10, epochs=100)'''

'mlp_keras.fit(X_df_train, y_df_train, batch_size=10, epochs=100)'

In [27]:
#predict_mlp_keras = mlp_keras.predict(X_df_test)

In [28]:
#predict_mlp_keras = (predict_mlp_keras > 0.5)

In [29]:
#accuracy_score(y_df_test, predict_mlp_keras)

In [30]:
#confusion_matrix(y_df_test, predict_mlp_keras)

In [31]:
#print(classification_report(y_df_test, predict_mlp_keras ))

In [32]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(verbose=True, max_iter=1000, tol=0.000010, hidden_layer_sizes=(100, 100))
mlp.fit(X_df_train, y_df_train)

Iteration 1, loss = 0.37726211
Iteration 2, loss = 0.32186687
Iteration 3, loss = 0.31141765
Iteration 4, loss = 0.30399089
Iteration 5, loss = 0.29862869
Iteration 6, loss = 0.29460698
Iteration 7, loss = 0.29163226
Iteration 8, loss = 0.28734696
Iteration 9, loss = 0.28483126
Iteration 10, loss = 0.28341999
Iteration 11, loss = 0.28030077
Iteration 12, loss = 0.27810648
Iteration 13, loss = 0.27394857
Iteration 14, loss = 0.27267237
Iteration 15, loss = 0.26867806
Iteration 16, loss = 0.26699974
Iteration 17, loss = 0.26427733
Iteration 18, loss = 0.26176899
Iteration 19, loss = 0.25985553
Iteration 20, loss = 0.25590494
Iteration 21, loss = 0.25496811
Iteration 22, loss = 0.25211283
Iteration 23, loss = 0.24918829
Iteration 24, loss = 0.24734152
Iteration 25, loss = 0.24465921
Iteration 26, loss = 0.24354974
Iteration 27, loss = 0.23997196
Iteration 28, loss = 0.23830033
Iteration 29, loss = 0.23562037
Iteration 30, loss = 0.23209469
Iteration 31, loss = 0.23229862
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=1e-05, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [33]:
predict_mlp = mlp.predict(X_df_test)

In [34]:
accuracy_score(y_df_test, predict_mlp)

0.8151484135107472

In [35]:
confusion_matrix(y_df_test, predict_mlp)

array([[3260,  433],
       [ 470,  722]])

In [36]:
print(classification_report(y_df_test, predict_mlp))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88      3693
           1       0.63      0.61      0.62      1192

    accuracy                           0.82      4885
   macro avg       0.75      0.74      0.75      4885
weighted avg       0.81      0.82      0.81      4885



In [37]:
kfold = KFold(n_splits=10, shuffle=True)
score_mlp = cross_val_score(mlp, X_df, y_df, cv = kfold)
score_mlp.mean()

Iteration 1, loss = 0.36347327
Iteration 2, loss = 0.32193810
Iteration 3, loss = 0.31005543
Iteration 4, loss = 0.30355876
Iteration 5, loss = 0.29873867
Iteration 6, loss = 0.29359691
Iteration 7, loss = 0.29080275
Iteration 8, loss = 0.28755749
Iteration 9, loss = 0.28460970
Iteration 10, loss = 0.28232195
Iteration 11, loss = 0.27945129
Iteration 12, loss = 0.27766088
Iteration 13, loss = 0.27497488
Iteration 14, loss = 0.27229352
Iteration 15, loss = 0.26949178
Iteration 16, loss = 0.26807114
Iteration 17, loss = 0.26519590
Iteration 18, loss = 0.26283280
Iteration 19, loss = 0.26062176
Iteration 20, loss = 0.25958707
Iteration 21, loss = 0.25556274
Iteration 22, loss = 0.25414328
Iteration 23, loss = 0.25220389
Iteration 24, loss = 0.24947748
Iteration 25, loss = 0.24727533
Iteration 26, loss = 0.24593682
Iteration 27, loss = 0.24345768
Iteration 28, loss = 0.24119102
Iteration 29, loss = 0.23878119
Iteration 30, loss = 0.23850622
Iteration 31, loss = 0.23508201
Iteration 32, los

0.8208595604703988

In [42]:
#hold
mlp_hold = mlp.fit(X_df, y_df)

Iteration 1, loss = 0.37087089
Iteration 2, loss = 0.31990380
Iteration 3, loss = 0.30935038
Iteration 4, loss = 0.30263992
Iteration 5, loss = 0.29866038
Iteration 6, loss = 0.29465254
Iteration 7, loss = 0.29158229
Iteration 8, loss = 0.29002726
Iteration 9, loss = 0.28587846
Iteration 10, loss = 0.28354505
Iteration 11, loss = 0.28164010
Iteration 12, loss = 0.27894465
Iteration 13, loss = 0.27679635
Iteration 14, loss = 0.27511571
Iteration 15, loss = 0.27264200
Iteration 16, loss = 0.26913299
Iteration 17, loss = 0.26806864
Iteration 18, loss = 0.26528629
Iteration 19, loss = 0.26287101
Iteration 20, loss = 0.26102884
Iteration 21, loss = 0.25767552
Iteration 22, loss = 0.25577936
Iteration 23, loss = 0.25358864
Iteration 24, loss = 0.25297871
Iteration 25, loss = 0.25008240
Iteration 26, loss = 0.24775644
Iteration 27, loss = 0.24504887
Iteration 28, loss = 0.24305110
Iteration 29, loss = 0.24009771
Iteration 30, loss = 0.23946210
Iteration 31, loss = 0.23733711
Iteration 32, los

In [44]:
#DataFrame
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']

df = pd.read_csv('adult.test.csv', names=headers)

#Tratamento de dados faltantes para o mais representatívo
columns = df.columns
for i in columns:
    missing = df[i].isin([' ?']).sum()
    df[i] = df[i].replace(' ?', np.NaN)
df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))

#Divisão em Parâmetro e Classe
X_df = df.iloc[:, 0:14].values
y_df = df.iloc[:, 14].values

#LabelEncoder
def labelencoder(pd_serie):
    labelencoder = LabelEncoder()
    pd_serie = labelencoder.fit_transform(pd_serie)
    return pd_serie

X_df[:, 1] = labelencoder(X_df[:, 1])
X_df[:, 3] = labelencoder(X_df[:, 3])
X_df[:, 5] = labelencoder(X_df[:, 5])
X_df[:, 6] = labelencoder(X_df[:, 6])
X_df[:, 7] = labelencoder(X_df[:, 7])
X_df[:, 8] = labelencoder(X_df[:, 8])
X_df[:, 9] = labelencoder(X_df[:, 9])
X_df[:, 13] = labelencoder(X_df[:, 13])
y_df = labelencoder(y_df)

#One Hot Encoder
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])], remainder='passthrough')
X_df = onehotencorder.fit_transform(X_df).toarray()

#Standard Scaler
scaler = StandardScaler()
X_df = scaler.fit_transform(X_df)

In [46]:
#knn
predict_knn_hold = knn_hold.predict(X_df)

ValueError: ignored

In [48]:
#random_forest
predict_random_forest_hold = random_forest_hold.predict(X_df)

ValueError: ignored

In [49]:
#MLP
predict_mlp_hold = mlp_hold.predict(X_df)

ValueError: ignored